In [17]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import TextLoader# Permite carregar arquivos
from langchain_text_splitters import CharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain import hub

c:\Users\Victor Bellé\Desktop\Projetos Python\RAG_LLM_Intro\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
load_dotenv(find_dotenv())
COHERE_API_KEY =os.getenv("COHERE_API_KEY")
PINECONE_API_KEY =os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [29]:

loader = TextLoader("data\mediublog.txt")
documents = loader.load()

<>:3: SyntaxWarning: invalid escape sequence '\m'
<>:3: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Victor Bellé\AppData\Local\Temp\ipykernel_29204\944228328.py:3: SyntaxWarning: invalid escape sequence '\m'
  loader = TextLoader("data\mediublog.txt")


In [30]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
print(f"Total de partes criadas: {len(docs)}")


Total de partes criadas: 2


In [32]:
from langchain_cohere import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

embeddings = CohereEmbeddings(model="embed-english-v3.0", cohere_api_key=COHERE_API_KEY)
index_name = "rag-test"

vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs, index_name=index_name, embedding=embeddings
)

In [34]:
print("Fatiamento do documento em menores chunks ")

text_splitter = CharacterTextSplitter(
    chunk_size =1000,
    chunk_overlap = 0
)
docs = text_splitter.split_documents(documents)

Fatiamento do documento em menores chunks 


In [36]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings
)

In [37]:
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

In [39]:
query = "What is Vector Store?"
vectorstore.similarity_search(query, k=1)

[Document(id='8642cd93-54a2-43fe-910f-700d22f93ee3', metadata={'source': 'data\\mediublog.txt'}, page_content='In recent months, weâ€™ve witnessed an explosion of interest in multi-agent systems powered by LLM.\n\nThe excitement surrounding these systems stems from their impressive ability to decompose complex tasks and work collaboratively to address complex problems.\n\nHowever, as developers rush to adopt and deploy MAS-based solutions, a critical aspect remains under-discussed: security.\n\nMulti-agent systems introduce unique security challenges that go beyond those of single LLMs. By design, a multi-agent system consists of multiple LLM-powered agents interacting with each other, making autonomous decisions, accessing external data and tools, and possibly generating/executing code. All these aspects introduced expanded attack surfaces.\n\nIn this blog, letâ€™s explore the security landscape of LLM-powered multi-agent systems through a practical lens. We aim to answer three questi

In [42]:
llm = ChatGroq(
    model = "Gemma2-9b-It", # MODELO DE LLM UTILIZADO
    groq_api_key = GROQ_API_KEY, # CHAVE DE API DO GROQ
    temperature = 0.1, # TEMPERATURA DO LLM
)

### ** Carregar um prompt "padrão" via langchain hub **


In [43]:
from langchain.chains.combine_documents import create_stuff_documents_chain
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain   = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)

c:\Users\Victor Bellé\Desktop\Projetos Python\RAG_LLM_Intro\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [46]:
from langchain.chains import create_retrieval_chain
qa = create_retrieval_chain(
    vectorstore.as_retriever(),
    combine_docs_chain
)

In [48]:
res = qa.invoke({"input": "Why are multi-agent systems inherently vulnerable to cybersecurity attacks?"})
print(res['answer'])

Multi-agent systems are inherently vulnerable to cybersecurity attacks because:

* **They are composed of multiple LLM-powered agents:** This means there are more points of potential vulnerability compared to a single LLM.
* **Agents interact with each other:**  This interaction creates opportunities for malicious agents to influence or manipulate other agents.
* **Agents make autonomous decisions:**  This autonomy can be exploited if an agent is compromised, as it may make decisions that are harmful to the system or its users.
* **Agents access external data and tools:** This expands the attack surface, as vulnerabilities in external systems can be leveraged to attack the multi-agent system.
* **Agents may generate and execute code:** This introduces the risk of code injection attacks, where malicious code is inserted into the system. 


All these factors combine to create a more complex and vulnerable security landscape compared to single LLM systems. 

